In [36]:
import json
import nltk
import re
import csv
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.linear_model import LogisticRegression
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import classification_report
import numpy as np
import pandas as pd

pd.set_option('display.max_colwidth', 300)

# Read data
df = pd.read_pickle('Data\MovieSummaries\plots_genres_reduced_to_60.pkl')

# Preprocessing functions
def clean_text(text):
    text = re.sub("\'", "", text)
    text = re.sub("[^a-zA-Z]", " ", text)
    text = " ".join(text.split())
    text = text.lower()
    return text

def remove_stopwords(text):
    no_stop = []
    for i in text.split():
        if i not in stopwords:
            no_stop.append(i)
    return " ".join(no_stop)

def freq_plot(text):
    
    words = " ".join([x for x in text])
    words = words.split()
    fdist = nltk.FreqDist(words)
    return fdist



In [37]:

# genre = []

# for i in data["genre"]:
#     #print(list(json.loads(i).values()))
#     genre.append(list(json.loads(i).values()))

# data["genre_new"] = genre

# data_new = data[~(data["genre_new"].str.len() == 0)]

# dumm = []

# for i in genre:
#     for j in i:
#         #print(j)
#         dumm.append(j)
# all_genre = list(set(dumm))


# genre_new = nltk.FreqDist(dumm)
# genre_df = pd.DataFrame.from_dict(genre_new, orient="index")
# genre_df.columns = ["Count"]
# genre_df = genre_df.reset_index()
# genre_df.columns = ["Genre", "Count"]


# #plt.figure(figsize=(12,12))
# #sns.barplot(data=genre_df.sort_values("Count", ascending=False).loc[:20, :], x="Count", y="Genre")

In [38]:
# #------------------#
# #Delte film with only one genre and this genre is less than 10 films

# #film per genre
# film_per_genre = pd.DataFrame(columns=["Genre", "Count"])
# film_per_genre["Genre"] = all_genre

# for i in range(len(film_per_genre)):
#     film_per_genre["Count"][i] = len(data_new[data_new["genre_new"].apply(lambda x : film_per_genre["Genre"][i] in x)])

# film_per_genre = film_per_genre.sort_values("Count", ascending=False)
# film_per_genre = film_per_genre.reset_index(drop=True)
# print(film_per_genre)

# #delete all film from data_new with only one genre and this genre is less than 10 films
# data_new = data_new[data_new["genre_new"].apply(lambda x : (len(x) > 1) & (film_per_genre[film_per_genre["Genre"] == x[0]]["Count"].values[0] > 100))]

# #clean text
# data_new["clean_plot"] = data_new["plot"].apply(lambda x : clean_text(x))

In [39]:
df.reset_index(inplace=True)
df

,index,id,text,title,genre
0,0,330,in order to prepare the role of an important old actress a theatre student interviews three actresses who were her pupils an international diva a television star and a dubbing director,Actrius,"[drama, comedydrama]"
1,1,3217,after being pulled through a time portal ash williams lands in ad where he is almost immediately captured by lord arthurs men who suspect him to be an agent for duke henry with whom arthur is at war he is enslaved along with the captured henry his gun and chainsaw confiscated and is taken to a c...,Army of Darkness,"[cult, horror, drama, actionadventure, fantasy, horror, comedy, action, comedy]"
2,2,3333,the film follows two juxtaposed families the northern stonemans consisting of the abolitionist congressman austin stoneman his two sons and his daughter elsie and the southern camerons a family including two daughters margaret and flora and three sons most notably ben the stoneman brothers visi...,The Birth of a Nation,"[silent, indie, drama, blackandwhite, drama, war]"
3,3,3746,hatnote in los angeles november retired police officer rick deckard is detained by officer gaff and brought to meet with his former supervisor bryant deckard whose job as a blade runner was to track down bioengineered beings known as replicants and retire them is told by bryant that several have...,Blade Runner,"[thriller, science fiction, cult, drama, noir, crime, chinese]"
4,4,3837,in the american old west of construction on a new railroad led by lyle runs into quicksand the route has to be changed which will require it to go through rock ridge a frontier town where everyone has the last name of johnson the conniving state attorney general hedley lamarr wants to buy the la...,Blazing Saddles,"[western, satire, comedy]"
...,...,...,...,...,...
41572,41788,37373877,according to horrorfest online six people with a common past converge years later at the funeral of an old friend while putting the affairs in order a map is presented to them they follow the directions to find a longlost time capsule from their youth the search is requested by the dead friend i...,Crazy Eights,"[cult, horror]"
41573,41789,37473592,thoppul kodihttpblogspotinthoppulkodihtml proclaims the pathetic life about the cylon refugees music done by music director thomas rathnamhttpwwwflipkartcomthoppulkodip he was also known as isai aruvi thomas rathnam he is an upcoming music composer from india he has done tamil films karuvarai po...,Thoppul Kodi,[drama]
41574,41790,37478048,anand verma a widower and father of a child admits a woman in the hospital who had lost her memory since he admitted her in the hospital the doctor asks him to take care of her until she recovers from amnesia the doctor names her asha which is the name of anands wife and makes her believe that s...,Mr. Bechara,[comedy]
41575,41791,37492363,when clovers childhood friend cherries returns to his hometown after a ten year separation he finds himself immediately attracted to her still outgoing spontaneous personality it isnt long before their rekindled friendship develops into a sweet quirky romance but as time passes and passions fade...,Cherries and Clover,"[comedy, drama, romance]"


In [40]:
#clean text
df["text"] = df["text"].apply(lambda x : clean_text(x))

# Get stopwords
from nltk.corpus import stopwords
stopwords = set(stopwords.words("english"))
df["text"] = df["text"].apply(lambda x : remove_stopwords(x))

# Binarize genres
from sklearn.preprocessing import MultiLabelBinarizer
multilabel_bina = MultiLabelBinarizer()
multilabel_bina.fit(df["genre"])
y = multilabel_bina.transform(df["genre"])

# TF-IDF Vectorizer
tfidf_vect = TfidfVectorizer(max_df= 0.8, max_features=10000)
xtrain, xval, ytrain, yval = train_test_split(df["text"], y, test_size = 0.2, random_state= 9)
xtrain_tfidf = tfidf_vect.fit_transform(xtrain)
xval_tfidf = tfidf_vect.transform(xval)

In [41]:
#make list aff all genre
genre_list = []
for i in range(len(df["genre"])):
    genre_list.append(df["genre"].iloc[i][0])

#unique genre
unique_genre = list(set(genre_list))
len(unique_genre)

60

In [42]:
# Documents Example with 5 Plots
documents = df["text"][0:5].reset_index(drop=True)

# Term Frequency-Inverse Document Frequency vectorizer
Tfid_vect = TfidfVectorizer()

# Transform the documents using the vectorizer
documents_vect = Tfid_vect.fit_transform(documents)

# Convert the transformed for better visability. Normally sparse matrix
#df = pd.DataFrame(documents_vect.toarray(), columns=Tfid_vect.get_feature_names_out())

#df

# Multi label logistic Regression

In [43]:

# Logistic Regression
logistic_mod = LogisticRegression()

# Separate binary classifier for each class label for multi-label classification
onevsall = OneVsRestClassifier(logistic_mod) 

# Train model
onevsall.fit(xtrain_tfidf, ytrain)

# Predict and evaluate
y_pred = onevsall.predict(xval_tfidf)
print(classification_report(yval, y_pred))

# Sample predictions
def new_val(x):  
    x = remove_stopwords(clean_text(x))
    x_vec = tfidf_vect.transform([x])
    x_pred = onevsall.predict(x_vec)
    return multilabel_bina.inverse_transform(x_pred)


c:\Users\anna-\anaconda3\envs\NLP_project\Lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


              precision    recall  f1-score   support

           0       0.64      0.26      0.37      1164
           1       0.64      0.13      0.21       690
           2       0.00      0.00      0.00       235
           3       0.68      0.16      0.25       677
           4       0.76      0.26      0.39       498
           5       0.00      0.00      0.00        63
           6       0.00      0.00      0.00       135
           7       0.00      0.00      0.00       119
           8       0.00      0.00      0.00        76
           9       0.91      0.09      0.17       789
          10       0.00      0.00      0.00        71
          11       0.58      0.09      0.15       203
          12       0.00      0.00      0.00        54
          13       0.00      0.00      0.00       138
          14       0.00      0.00      0.00       159
          15       0.88      0.22      0.35       195
          16       0.71      0.43      0.54      2505
          17       0.00    

c:\Users\anna-\anaconda3\envs\NLP_project\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\anna-\anaconda3\envs\NLP_project\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [50]:
for i in range(5): 
    k = xval.sample(1).index[0] 
    print("Movie: ", df['title'][k], "\nPredicted genre: ", new_val(xval[k]))
    print("Actual genre: ",df['genre'][k], "\n")

Movie:  Mallu Singh 
Predicted genre:  [('drama',)]
Actual genre:  ['comedy'] 

Movie:  Detachment 
Predicted genre:  [('drama',)]
Actual genre:  ['drama'] 

Movie:  Paroma 
Predicted genre:  [('drama',)]
Actual genre:  ['drama'] 

Movie:  Goof on the Roof 
Predicted genre:  [('comedy',)]
Actual genre:  ['short', 'comedy', 'blackandwhite'] 

Movie:  Woman Hater 
Predicted genre:  [('comedy', 'drama', 'romance')]
Actual genre:  ['comedy'] 



In [51]:
for i in range(5): 
    k = xval.sample(1).index[0] 
    print("Movie: ", df['title'][k], "\nPredicted genre: ", new_val(xval[k]))
    print("Actual genre: ",df['genre'][k], "\n")

Movie:  True Women For Sale 
Predicted genre:  [('chinese', 'war')]
Actual genre:  ['chinese'] 

Movie:  Devdas 
Predicted genre:  [('drama', 'war')]
Actual genre:  ['war', 'musical', 'drama', 'drama', 'romance', 'bollywood'] 

Movie:  Sinners in Paradise 
Predicted genre:  [('drama',)]
Actual genre:  ['drama', 'adventure'] 

Movie:  El anónimo 
Predicted genre:  [('drama',)]
Actual genre:  ['drama'] 

Movie:  The Parallax View 
Predicted genre:  [('action', 'crime', 'drama', 'thriller')]
Actual genre:  ['thriller', 'psychological thriller', 'drama', 'suspense', 'political', 'political'] 



In [55]:
for i in range(5): 
    print(k)
    k = xval.sample(1).index[0] 
    print("Movie: ", df['title'][k], "\nPredicted genre: ", new_val(xval[k]))
    print("Actual genre: ",df['genre'][k], "\n")

12801
Movie:  Girls' Dormitory 
Predicted genre:  [('drama', 'romance')]
Actual genre:  ['drama', 'romance', 'drama'] 

32632
Movie:  Sky High 
Predicted genre:  [('horror',)]
Actual genre:  ['thriller', 'horror', 'war', 'supernatural', 'japanese', 'actionadventure', 'mystery', 'fantasy', 'drama', 'crime', 'action'] 

8646
Movie:  Cozy Dens 
Predicted genre:  [('drama', 'war')]
Actual genre:  ['family', 'comedydrama', 'period piece', 'drama', 'comedy', 'coming of age'] 

24471
Movie:  They Came to Blow Up America 
Predicted genre:  [('action',)]
Actual genre:  ['spy', 'action', 'drama', 'war'] 

33683
Movie:  Little Indian, Big City 
Predicted genre:  [('drama', 'romance')]
Actual genre:  ['romance', 'adventure', 'comedy'] 



In [62]:
for i in range(5): 
    print(k)
    k = xval.sample(1).index[0] 
    print("Movie: ", df['title'][k], "\nPredicted genre: ", new_val(xval[k]))
    print("Actual genre: ",df['genre'][k], "\n")

1640
Movie:  The Nightmare Before Christmas 
Predicted genre:  [('comedy', 'family')]
Actual genre:  ['childrensfamily', 'creature', 'musical', 'animation', 'fantasy', 'childrens', 'family'] 

1538
Movie:  Betty Boop's Hallowe'en Party 
Predicted genre:  [('comedy', 'short')]
Actual genre:  ['short', 'family', 'horror', 'musical', 'animation'] 

14716
Movie:  Querelle 
Predicted genre:  [('drama',)]
Actual genre:  ['lgbt', 'drama', 'war', 'gay', 'gay', 'drama', 'gay', 'drama', 'romance'] 

927
Movie:  It's Trad, Dad! 
Predicted genre:  [('comedy',)]
Actual genre:  ['musical', 'comedy', 'blackandwhite', 'comedy'] 

16109
Movie:  En Kadamai 
Predicted genre:  [('drama',)]
Actual genre:  ['drama'] 

